For changing formats and wrangling training data.

In [ ]:
import csv
import utils
import pathlib

In [ ]:
exp_dir = pathlib.Path("..")

Turning our own matches into same format as given:

In [ ]:
index = 0
for fn in ("matches", "notmatches"):
    with (exp_dir / (fn + ".csv")).open("r") as fin, \
        (utils.datadir / "links" / (fn + ".csv")).open("w") as fout:
        reader = csv.reader(fin)
        writer = csv.writer(fout, delimiter="|")
        writer.writerow(["linkID", "kilde", "kipnr", "lbnr",
                         "tabel", "navn", "faarb"])
        while True:
            try:
                a = next(reader)
                b = next(reader)
                linkID = "a" + str(index)
                for row in (a,b):
                    ft = row[1]
                    tabel = row[2]
                    kip = row[-2]
                    løb = row[-1]
                    if "," in løb:
                        løb = løb.split(",")[0]
                    navn = row[5]
                    fødeår = row[8]
                    writer.writerow([linkID, ft, kip, løb,
                                    tabel, navn, fødeår])
                index += 1
            except StopIteration:
                break

Reading from such a file into something (being reconstructed presently)...

# To practical format

In [ ]:
header = "a_FT a_Kipnr a_Løbenr b_FT b_Kipnr b_Løbenr".split()

In [ ]:
relevant = {1845, 1850, 1860, 1880, 1885}
keys = {}
with (utils.datadir / "links" / "given_links.csv").open("r", encoding="latin1") as fd:
    reader = csv.reader(fd, delimiter="|")
    next(reader)
    for row in reader:
        ft = utils.extractYear(row[1])
        if ft not in relevant:
            continue
        løbenr = int(float(row[3])) if "." in row[3] else int(row[3])
        key = (ft,row[2],løbenr)
        keys.setdefault(row[0], []).append(key)    

In [ ]:
keys

In [ ]:
with (utils.datadir / "links" / "given.csv").open("w", encoding="utf-8") as fd:
    writer = csv.writer(fd, delimiter="|", lineterminator="\n")
    writer.writerow(header)
    for l in keys.values():
        if len(l) < 2:
            continue
        l.sort()
        for (a, b) in zip(l, l[1:]):
            writer.writerow(a + b)

In [ ]:
d = csv.get_dialect("unix")

In [ ]:
d.lineterminator

In [ ]:
keys

In [ ]:


def buildLinksDict(fd):
    reader = csv.reader(fd, delimiter="|")
    next(reader) # skip header
    links = {year: {} for year in relevant} # year -> (kip,løb) -> linkID
    for row in reader:
        year = utils.extractYear(row[1])
        if year is None:
            continue
        if year not in relevant:
            continue
        kipløb = (row[2], int(row[3]))
        links[year][kipløb] = row[0]
    return links

In [ ]:
with (utils.datadir / "links" / "matches.csv").open("r") as fd:
    d = buildLinksDict(fd)

In [ ]:
d

In [ ]:
with (utils.datadir / "links" / "matches.csv").open("r") as fd:
    reader = csv.reader(fd, delimiter="|")
    next(reader)
    data = []
    for row in reader:
        for i in (0,2,3,5):
            row[i] = int(row[i])
            data.append(row)

In [ ]:
data

In [ ]:
with (utils.datadir / "links" / "matches.csv").open("w") as fd:
    writer = csv.writer(fd, dialect="unix")
    writer.writerow(header)
    writer.writerows(data)